In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print(f"CUDA_VISIBLE_DEVICES set to: {os.environ.get('CUDA_VISIBLE_DEVICES')}")

import torch
import torch.nn as nn
from transformers import AutoModel

In [ ]:
from datasets import load_dataset


dataset_name = "FinanceMTEB/financial_phrasebank"

print(f"Loading {dataset_name} dataset")

try:
    raw_dataset = load_dataset(dataset_name)

    print("\nDataset loaded successfully!")
    print("\nDataset structure:")
    print(raw_dataset)


    if 'train' in raw_dataset:
        print("\n Some examples sentences from the dataset:")
        print(raw_dataset['train'].shuffle(seed=42).select(range(5)))
    else:
        print("\nNote: Dataset doesn't contain a 'train' split. Showing the available splits.")

except Exception as e:
    print(f"\nFailed to load the dataset: {e}")
    print("Double-checking the dataset name and structure on the Hugging Face Hub.")

In [ ]:
from transformers import AutoTokenizer


MODEL_CHECKPOINT = "distilbert-base-uncased"
print(f"Loading tokenizer for model: {MODEL_CHECKPOINT} ")

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

print("Tokenizer loaded.")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

print("\nTokenizing the dataset: ")

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
print("Tokenization is complete.")


tokenized_datasets = tokenized_datasets.remove_columns(["text", "label_text"])


tokenized_datasets = tokenized_datasets.rename_column("label", "labels")


tokenized_datasets.set_format("torch")

print("\nDataset cleaned and prepared for training.")
print("\nFinal dataset structure:")
print(tokenized_datasets)


print("\nExample of a processed training example:")
print(tokenized_datasets["train"][0])

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

all_labels = tokenized_datasets["train"]["labels"].numpy()

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(all_labels),
    y=all_labels
)

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

print(f"Calculated Class Weights: {class_weights}")
print(f"Class Weights Tensor on device {class_weights_tensor.device}: {class_weights_tensor}")


class CustomSentimentClassifierWithWeights(nn.Module):
    def __init__(self, base_model, class_weights, num_labels=3, dropout_rate=0.1):
        super().__init__()
        self.num_labels = num_labels
        self.base_model = base_model
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(768, num_labels)

        self.loss_fct = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0]
        dropped_output = self.dropout(cls_output)
        logits = self.classifier(dropped_output)

        loss = None
        if labels is not None:
            loss = self.loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

print("\nRe-instantiating custom model with class weights...")
model = CustomSentimentClassifierWithWeights(
    base_model=base_model,
    class_weights=class_weights_tensor,
    num_labels=3
)

model.to(device)
print(f"Model moved to: {device}")

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics_detailed(eval_pred):
    """
    Calculates accuracy, weighted avg F1/precision/recall,
    macro avg F1/precision/recall, and per-class F1/precision/recall.
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate overall accuracy
    acc = accuracy_score(labels, predictions)

    # Calculate weighted metrics
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted', zero_division=0
    )

    # Calculate macro metrics (unweighted average)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        labels, predictions, average='macro', zero_division=0
    )

    # Calculate per-class metrics
    precision_per_class, recall_per_class, f1_per_class = precision_recall_fscore_support(
        labels, predictions, average=None, zero_division=0
    )

    labels_map = {-1: "neg", 0: "neu", 1: "pos"}

    metrics = {
        'accuracy': acc,
        'f1_weighted': f1_weighted,
        'precision_weighted': precision_weighted,
        'recall_weighted': recall_weighted,
        'f1_macro': f1_macro,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
    }

    for i, label_id in enumerate(labels_map):
        metrics[f'f1_{labels_map[label_id]}'] = f1_per_class[i]
        metrics[f'precision_{labels_map[label_id]}'] = precision_per_class[i]
        metrics[f'recall_{labels_map[label_id]}'] = recall_per_class[i]

    return metrics

print("Detailed compute_metrics function defined.")

In [ ]:
from transformers import TrainingArguments, SchedulerType

output_dir = "finbert-transfer-learned-model"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    push_to_hub=False,
    logging_strategy="steps",
    logging_steps=50,
    lr_scheduler_type=SchedulerType.LINEAR,
    warmup_steps=50,
)

print("TrainingArguments defined with LR scheduler and macro F1.")

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    """
    Calculates accuracy, F1, precision, and recall using only scikit-learn.
    Uses a weighted average for precision, recall, and F1.
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted', zero_division=0)

    acc = accuracy_score(labels, predictions)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

print("Corrected compute_metrics function defined.")

In [ ]:
from transformers import Trainer


train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_detailed,
)

print("\nTrainer instantiated. Starting training...")


trainer.train()

print("\nTraining complete!")